In [56]:
#import dependencies
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import re
from decimal import Decimal


In [57]:
#read the CSV == source - https://www.kaggle.com/datasets/patkle/coffeereviewcom-over-7000-ratings-and-reviews
csv_data = "reviews_feb_2023.csv"
coffee_df = pd.read_csv(csv_data)
coffee_df.head()

,title,rating,acidity_structure,aftertaste,aroma,body,flavor,with_milk,agtron,blind_assessment,bottom_line,coffee_origin,est_price,notes,review_date,roast_level,roaster,roaster_location,url
0,Bolivia Manantial Gesha,93,9.0,8.0,9,8,9,NaN,60/78,"Richly aromatic, floral-toned. Magnolia, cocoa...",This washed Boliva Gesha has all the aromatics...,"Caranavi, Bolivia",$30.00/12 ounces,"Produced by Angel Mamani Chambi, entirely of t...",Jan-23,Medium-Light,Red Rooster Coffee Roaster,"Floyd, Virginia",https://www.coffeereview.com/review/bolivia-ma...
1,Yellow Pacamara Carbonic Maceration Nanolot,92,8.0,8.0,9,8,9,NaN,60/78,"Crisply sweet-savory. White peach, hop flowers...",A carbonic-macerated Yellow Pacamara grown in ...,"Matagalpa growing region, Nicaragua",$160.00/12 ounces,Produced by Benjamin Weiner at Finca Idealista...,Dec-22,Medium-Light,Eccentricity Coffee Co.,"Cleveland, Ohio",https://www.coffeereview.com/review/brix-break...
2,Ethiopia Gera Genji Challa,94,9.0,8.0,9,9,9,NaN,59/77,"Delicately aromatic, complex. Lilac, cocoa nib...","A nuanced, multilayered washed Ethiopia cup wi...","Agaro Gera, Jimma Zone, Oromia State, Ethiopia",$28.00/12 ounces,Ethiopia coffees like this one are largely pro...,Dec-22,Medium-Light,Mostra Coffee,"San Diego, California",https://www.coffeereview.com/review/ethiopia-g...
3,Yirgacheffe Mengesha Natural,94,9.0,8.0,9,9,9,NaN,60/77,"High-toned, fruit-driven. Boysenberry, pear, c...",A fruit medley in a cup — think boysenberry an...,"Yirgacheffe growing region, southern Ethiopia",$20.50/12 ounces,Produced at Mengesha Farm from selections of i...,Nov-22,Medium-Light,Regent Coffee,"Glendale, California",https://www.coffeereview.com/review/yirgacheff...
4,Tropical Summer Colombia La Sierra,93,9.0,8.0,9,8,9,NaN,60/77,"Fruit-driven, crisply chocolaty. Goji berry, d...","An experimentally processed Colombia, sweetly ...","La Sierra, Cauca Department, Colombia",$18.99/8 ounces,Produced by smallholding farmers from trees of...,Nov-22,Medium-Light,Merge Coffee Company,"Harrisonburg, Virginia",https://www.coffeereview.com/review/tropical-s...


In [58]:
#check for null values
coffee_df.isnull().sum()

title                   0
rating                  0
acidity_structure    4875
aftertaste            872
aroma                  50
body                   11
flavor                 16
with_milk            6044
agtron                  0
blind_assessment        1
bottom_line          4080
coffee_origin         505
est_price            2039
notes                   8
review_date             0
roast_level           374
roaster                 0
roaster_location        3
url                     0
dtype: int64

In [59]:
#drop the unneccessary columns 'with_milk','agtron','roaster','url' and high null value cols which are 'acidity_structure', 'bottom_line'
coffee1_df = coffee_df.drop(columns=['with_milk','agtron','roaster','url','acidity_structure', 'bottom_line', "review_date"])
coffee1_df.head()
coffee1_df.index      

RangeIndex(start=0, stop=7041, step=1)

In [60]:
#check if cols are dropped and detect which cols have null values
coffee1_df.isnull().sum()

title                  0
rating                 0
aftertaste           872
aroma                 50
body                  11
flavor                16
blind_assessment       1
coffee_origin        505
est_price           2039
notes                  8
roast_level          374
roaster_location       3
dtype: int64

In [64]:
#removed columns and null values to show 4904 rows × 12 columns with no null values and reset title to reorder
coffee2_df = coffee1_df.dropna(subset=['title', 'aftertaste', 'aroma', 'body', 'flavor', 'blind_assessment',
                                       'coffee_origin', 'est_price', 'notes', 'roast_level', 'roaster_location'])
coffee2_df = coffee2_df.drop_duplicates()
coffee2_df.set_index('title', inplace = True)
coffee2_df.reset_index().head()


,title,rating,aftertaste,aroma,body,flavor,blind_assessment,coffee_origin,est_price,notes,roast_level,roaster_location
0,Bolivia Manantial Gesha,93,8.0,9,8,9,"Richly aromatic, floral-toned. Magnolia, cocoa...","Caranavi, Bolivia",$30.00/12 ounces,"Produced by Angel Mamani Chambi, entirely of t...",Medium-Light,"Floyd, Virginia"
1,Yellow Pacamara Carbonic Maceration Nanolot,92,8.0,9,8,9,"Crisply sweet-savory. White peach, hop flowers...","Matagalpa growing region, Nicaragua",$160.00/12 ounces,Produced by Benjamin Weiner at Finca Idealista...,Medium-Light,"Cleveland, Ohio"
2,Ethiopia Gera Genji Challa,94,8.0,9,9,9,"Delicately aromatic, complex. Lilac, cocoa nib...","Agaro Gera, Jimma Zone, Oromia State, Ethiopia",$28.00/12 ounces,Ethiopia coffees like this one are largely pro...,Medium-Light,"San Diego, California"
3,Yirgacheffe Mengesha Natural,94,8.0,9,9,9,"High-toned, fruit-driven. Boysenberry, pear, c...","Yirgacheffe growing region, southern Ethiopia",$20.50/12 ounces,Produced at Mengesha Farm from selections of i...,Medium-Light,"Glendale, California"
4,Tropical Summer Colombia La Sierra,93,8.0,9,8,9,"Fruit-driven, crisply chocolaty. Goji berry, d...","La Sierra, Cauca Department, Colombia",$18.99/8 ounces,Produced by smallholding farmers from trees of...,Medium-Light,"Harrisonburg, Virginia"


In [65]:
#split roaster location and est price to new columns
coffee2_df[['city', 'state']] = coffee2_df['roaster_location'].str.split(',', n=1, expand=True)
coffee2_df[['price(usd)', 'weight']] = coffee2_df['est_price'].str.split('/', n=1, expand=True)
coffee3_df = coffee2_df.drop(columns=['roaster_location', 'est_price'])

coffee3_df.reset_index().head()

,title,rating,aftertaste,aroma,body,flavor,blind_assessment,coffee_origin,notes,roast_level,city,state,price(usd),weight
0,Bolivia Manantial Gesha,93,8.0,9,8,9,"Richly aromatic, floral-toned. Magnolia, cocoa...","Caranavi, Bolivia","Produced by Angel Mamani Chambi, entirely of t...",Medium-Light,Floyd,Virginia,$30.00,12 ounces
1,Yellow Pacamara Carbonic Maceration Nanolot,92,8.0,9,8,9,"Crisply sweet-savory. White peach, hop flowers...","Matagalpa growing region, Nicaragua",Produced by Benjamin Weiner at Finca Idealista...,Medium-Light,Cleveland,Ohio,$160.00,12 ounces
2,Ethiopia Gera Genji Challa,94,8.0,9,9,9,"Delicately aromatic, complex. Lilac, cocoa nib...","Agaro Gera, Jimma Zone, Oromia State, Ethiopia",Ethiopia coffees like this one are largely pro...,Medium-Light,San Diego,California,$28.00,12 ounces
3,Yirgacheffe Mengesha Natural,94,8.0,9,9,9,"High-toned, fruit-driven. Boysenberry, pear, c...","Yirgacheffe growing region, southern Ethiopia",Produced at Mengesha Farm from selections of i...,Medium-Light,Glendale,California,$20.50,12 ounces
4,Tropical Summer Colombia La Sierra,93,8.0,9,8,9,"Fruit-driven, crisply chocolaty. Goji berry, d...","La Sierra, Cauca Department, Colombia",Produced by smallholding farmers from trees of...,Medium-Light,Harrisonburg,Virginia,$18.99,8 ounces


In [67]:
#check for null values in new columns
coffee3_df.isnull().sum()

rating               0
aftertaste           0
aroma                0
body                 0
flavor               0
blind_assessment     0
coffee_origin        0
notes                0
roast_level          0
city                 0
state                8
price(usd)           0
weight              13
dtype: int64

In [68]:
#drop the null values in state and weight columns with null values
coffee3_df = coffee3_df.dropna(subset=['state', 'weight'])

coffee3_df.reset_index()

,title,rating,aftertaste,aroma,body,flavor,blind_assessment,coffee_origin,notes,roast_level,city,state,price(usd),weight
0,Bolivia Manantial Gesha,93,8.0,9,8,9,"Richly aromatic, floral-toned. Magnolia, cocoa...","Caranavi, Bolivia","Produced by Angel Mamani Chambi, entirely of t...",Medium-Light,Floyd,Virginia,$30.00,12 ounces
1,Yellow Pacamara Carbonic Maceration Nanolot,92,8.0,9,8,9,"Crisply sweet-savory. White peach, hop flowers...","Matagalpa growing region, Nicaragua",Produced by Benjamin Weiner at Finca Idealista...,Medium-Light,Cleveland,Ohio,$160.00,12 ounces
2,Ethiopia Gera Genji Challa,94,8.0,9,9,9,"Delicately aromatic, complex. Lilac, cocoa nib...","Agaro Gera, Jimma Zone, Oromia State, Ethiopia",Ethiopia coffees like this one are largely pro...,Medium-Light,San Diego,California,$28.00,12 ounces
3,Yirgacheffe Mengesha Natural,94,8.0,9,9,9,"High-toned, fruit-driven. Boysenberry, pear, c...","Yirgacheffe growing region, southern Ethiopia",Produced at Mengesha Farm from selections of i...,Medium-Light,Glendale,California,$20.50,12 ounces
4,Tropical Summer Colombia La Sierra,93,8.0,9,8,9,"Fruit-driven, crisply chocolaty. Goji berry, d...","La Sierra, Cauca Department, Colombia",Produced by smallholding farmers from trees of...,Medium-Light,Harrisonburg,Virginia,$18.99,8 ounces
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4878,Ethiopia Kemgin Espresso,92,8.0,9,7,9,Evaluated as espresso. In the small cup exhila...,Western Ethiopia,A special selection of hand sorted coffee from...,Medium,Portland,Oregon,$27.00,500 grams (17.6 oz.)
4879,Kenya AA Gandurine,93,8.0,9,8,9,Balanced in aroma and cup. Floral tones are co...,South-central Kenya.,Despite grower discontent and sporadic social ...,Medium,Lee,Massachusetts,$19.45,16 ounces
4880,Honduras David Mancia,94,9.0,9,8,9,"Deep, rich and pungent in aroma and cup. Round...","Santa Barbara Department, Honduras.",Produced on the two hectare farm of David Manc...,Medium-Light,Portland,Oregon,$10.50,250 grams (8.8 oz.)
4881,Doug E Fresh,90,8.0,9,7,8,"Gently and richly charred cedar, dark chocolat...",Not disclosed.,Founded in 1993 near the famous Winter Park sk...,Very Dark,Fraser,Colorado,$13.99,16 ounces


In [71]:
#change the datatype ---- need to integrate with Kim's code of price and weight (datattypes not changed)
coffee4_df = coffee3_df.astype({"rating":float, "aftertaste": float ,"aroma": float, "body": float, "flavor": float, "blind_assessment": "string", "coffee_origin": "string", 
                               "notes": "string", "roast_level": "string", "city": "string", "state": "string"})
#check for datatypes for any null values and datatypes
display(coffee4_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4883 entries, Bolivia Manantial Gesha to Tangletown Blend
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rating            4883 non-null   float64
 1   aftertaste        4883 non-null   float64
 2   aroma             4883 non-null   float64
 3   body              4883 non-null   float64
 4   flavor            4883 non-null   float64
 5   blind_assessment  4883 non-null   string 
 6   coffee_origin     4883 non-null   string 
 7   notes             4883 non-null   string 
 8   roast_level       4883 non-null   string 
 9   city              4883 non-null   string 
 10  state             4883 non-null   string 
 11  price(usd)        4883 non-null   object 
 12  weight            4883 non-null   object 
dtypes: float64(5), object(2), string(6)
memory usage: 534.1+ KB


None

In [74]:
#check if float columns have been standardized
coffee4_df.reset_index().head(3)

,title,rating,aftertaste,aroma,body,flavor,blind_assessment,coffee_origin,notes,roast_level,city,state,price(usd),weight
0,Bolivia Manantial Gesha,93.0,8.0,9.0,8.0,9.0,"Richly aromatic, floral-toned. Magnolia, cocoa...","Caranavi, Bolivia","Produced by Angel Mamani Chambi, entirely of t...",Medium-Light,Floyd,Virginia,$30.00,12 ounces
1,Yellow Pacamara Carbonic Maceration Nanolot,92.0,8.0,9.0,8.0,9.0,"Crisply sweet-savory. White peach, hop flowers...","Matagalpa growing region, Nicaragua",Produced by Benjamin Weiner at Finca Idealista...,Medium-Light,Cleveland,Ohio,$160.00,12 ounces
2,Ethiopia Gera Genji Challa,94.0,8.0,9.0,9.0,9.0,"Delicately aromatic, complex. Lilac, cocoa nib...","Agaro Gera, Jimma Zone, Oromia State, Ethiopia",Ethiopia coffees like this one are largely pro...,Medium-Light,San Diego,California,$28.00,12 ounces


In [ ]:
# !pip install forex_python
# from forex_python.converter import CurrencyRates